# Лекция 4. Создание базовых скриптов. Работа с файлами

# Создание базовых скриптов

Если говорить в целом, то скрипт - это обычный файл. В этом файле хранится последовательность команд, которые необходимо выполнить.

Начнем с базового скрипта. Выведем на стандартный поток вывода несколько строк.

Для этого надо создать файл access_template.py с таким содержимым:

In [ ]:
access_template = ['switchport mode access',
                   'switchport access vlan {}',
                   'switchport nonegotiate',
                   'spanning-tree portfast',
                   'spanning-tree bpduguard enable']

print('\n'.join(access_template).format(5))

Сначала элементы списка объединяются в строку, которая разделена символом `\n`, а в строку подставляется номер VLAN, используя форматирование строк.

После этого надо сохранить файл и перейти в командную строку.

## Исполняемый файл

Для того, чтобы файл был исполняемым, и не нужно было каждый раз писать python перед вызовом файла, нужно:

* сделать файл исполняемым (для Linux)
* в первой строке файла должна находиться строка `#!/usr/bin/env python{python version}`, в зависимости от того, какая версия Python используется по умолчанию

Пример файла access_template_exec.py:

In [ ]:
#!/usr/bin/env python3.10

access_template = ['switchport mode access',
                   'switchport access vlan {}',
                   'switchport nonegotiate',
                   'spanning-tree portfast',
                   'spanning-tree bpduguard enable']

print('\n'.join(access_template).format(5))

После этого:

```
chmod +x access_template_exec.py
```

Теперь можно вызывать файл таким образом:

```
$ ./access_template_exec.py
```

## Передача аргументов скрипту (argv)

Очень часто скрипт решает какую-то общую задачу. Например, скрипт обрабатывает как-то файл конфигурации. Конечно, в таком случае не хочется каждый раз руками в скрипте править название файла.

Гораздо лучше будет передавать имя файла как аргумент скрипта и затем использовать уже указанный файл.

Модуль sys позволяет работать с аргументами скрипта с помощью argv.

Пример access_template_argv.py:

In [ ]:
from sys import argv

interface = argv[1]
vlan = argv[2]

access_template = ['switchport mode access',
                   'switchport access vlan {}',
                   'switchport nonegotiate',
                   'spanning-tree portfast',
                   'spanning-tree bpduguard enable']

print('interface {}'.format(interface))
print('\n'.join(access_template).format(vlan))

Аргументы, которые были переданы скрипту, подставляются как значения в шаблон.

Тут надо пояснить несколько моментов:

* argv - это список
* все аргументы находятся в списке в виде строк
* argv содержит не только аргументы, которые передали скрипту, но и название самого скрипта

В данном случае в списке argv находятся такие элементы:

```
['access_template_argv.py', 'Gi0/7', '4']
```

Сначала идет имя самого скрипта, затем аргументы, в том же порядке.

# Работа с файлами

В реальной жизни для того чтобы полноценно использовать всё, что рассматривалось до этого раздела, надо разобраться как работать с файлами.

При работе с сетевым оборудованием (и не только), файлами могут быть:

* конфигурации (простые, не структурированные текстовые файлы)
    * работа с ними рассматривается в этом разделе
* шаблоны конфигураций
    * как правило, это какой-то специальный формат файлов.
* файлы с параметрами подключений
    * как правило, это структурированные файлы, в каком-то определенном формате: YAML, JSON, CSV
* другие скрипты Python

В этом разделе рассматривается работа с простыми текстовыми файлами. Например, конфигурационный файл Cisco.

В работе с файлами есть несколько аспектов:

* открытие/закрытие
* чтение
* запись


## Открытие файлов

Для начала работы с файлом, его надо открыть.

### `open`

Для открытия файлов, чаще всего, используется функция `open()`:

In [ ]:
file = open('file_name.txt', 'r')

В функции `open()`:

* `'file_name.txt'` - имя файла
* тут можно указывать не только имя, но и путь (абсолютный или относительный)
* `'r'` - режим открытия файла

Функция `open` создает объект file, к которому потом можно применять различные методы, для работы с ним.

Режимы открытия файлов:

* `r` - открыть файл только для чтения (значение по умолчанию)
* `r+` - открыть файл для чтения и записи
* `w` - открыть файл для записи
    * если файл существует, то его содержимое удаляется
    * если файл не существует, то создается новый
* `w+` - открыть файл для чтения и записи
    * если файл существует, то его содержимое удаляется
    * если файл не существует, то создается новый
* `a` - открыть файл для дополнения записи. Данные добавляются в конец файла
* `a+` - открыть файл для чтения и записи. Данные добавляются в конец файла

>
> r - read; a - append; w - write
>

## Чтение файлов

В Python есть несколько методов чтения файла:

* `read` - считывает содержимое файла в строку
* `readline` - считывает файл построчно
* `readlines` - считывает строки файла и создает список из строк

Посмотрим как считывать содержимое файлов, на примере файла r1.txt:

```
!
service timestamps debug datetime msec localtime show-timezone year
service timestamps log datetime msec localtime show-timezone year
service password-encryption
service sequence-numbers
!
no ip domain lookup
!
ip ssh version 2
!
```

### `read`

Метод `read` - считывает весь файл в одну строку.

Пример использования метода `read`:

In [ ]:
f = open('r1.txt')
f.read()

При повторном чтении файла отображается пустая строка. Так происходит из-за того, что при вызове метода `read`, считывается весь файл. И после того, как файл был считан, курсор остается в конце файла. Управлять положением курсора можно с помощью метода `seek`.

### `readline`

Построчно файл можно считать с помощью метода `readline`:

In [ ]:
f = open('r1.txt')

f.readline()

Но чаще всего проще пройтись по объекту file в цикле, не используя методы `read...`:

In [9]:
f = open('r1.txt')

for line in f:
    print(line)

!

service timestamps debug datetime msec localtime show-timezone year

service timestamps log datetime msec localtime show-timezone year

service password-encryption

service sequence-numbers

!

no ip domain lookup

!

ip ssh version 2

!


### `readlines`

Еще один полезный метод - `readlines`. Он считывает строки файла в список:

In [10]:
f = open('r1.txt')

f.readlines()

['!\n',
 'service timestamps debug datetime msec localtime show-timezone year\n',
 'service timestamps log datetime msec localtime show-timezone year\n',
 'service password-encryption\n',
 'service sequence-numbers\n',
 '!\n',
 'no ip domain lookup\n',
 '!\n',
 'ip ssh version 2\n',
 '!']

Если нужно получить строки файла, но без перевода строки в конце, можно воспользоваться методом `split` и как разделитель, указать символ `\n`:

Обратите внимание, что последний элемент списка - пустая строка.

Если перед выполнением `split`, воспользоваться методом `rstrip`, список будет без пустой строки в конце:

In [11]:
f = open('r1.txt')

f.read().rstrip().split('\n')

['!',
 'service timestamps debug datetime msec localtime show-timezone year',
 'service timestamps log datetime msec localtime show-timezone year',
 'service password-encryption',
 'service sequence-numbers',
 '!',
 'no ip domain lookup',
 '!',
 'ip ssh version 2',
 '!']

### `seek`

До сих пор, файл каждый раз приходилось открывать заново, чтобы снова его считать. Так происходит из-за того, что после методов чтения, курсор находится в конце файла. И повторное чтение возвращает пустую строку.

Чтобы ещё раз считать информацию из файла, нужно воспользоваться методом `seek`, который перемещает курсор в необходимое положение.

Пример открытия файла и считывания содержимого:

In [15]:
f = open('r1.txt')

print(f.read())
print('\'' + f.read() + '\'')
f.seek(0)
print(f.read())

!
service timestamps debug datetime msec localtime show-timezone year
service timestamps log datetime msec localtime show-timezone year
service password-encryption
service sequence-numbers
!
no ip domain lookup
!
ip ssh version 2
!
''
!
service timestamps debug datetime msec localtime show-timezone year
service timestamps log datetime msec localtime show-timezone year
service password-encryption
service sequence-numbers
!
no ip domain lookup
!
ip ssh version 2
!


## Запись файлов

При записи, очень важно определиться с режимом открытия файла, чтобы случайно его не удалить:

* `w` - открыть файл для записи. Если файл существует, то его содержимое удаляется
* `a` - открыть файл для дополнения записи. Данные добавляются в конец файла

При этом оба режима создают файл, если он не существует.

Для записи в файл используются такие методы:

* `write` - записать в файл одну строку
* `writelines` - позволяет передавать в качестве аргумента список строк

### `write`

Метод `write` ожидает строку, для записи.

Для примера, возьмем список строк с конфигурацией:

In [1]:
cfg_lines = ['!',
            'service timestamps debug datetime msec localtime show-timezone year',
            'service timestamps log datetime msec localtime show-timezone year',
            'service password-encryption',
            'service sequence-numbers',
            '!',
            'no ip domain lookup',
            '!',
            'ip ssh version 2',
            '!']

f = open('r2.txt', 'w')

cfg_lines_as_str = '\n'.join(cfg_lines)

f.write(cfg_lines_as_str)
f.close()

### `writelines`

Метод `writelines` ожидает список строк, как аргумент.

Запись списка строк cfg_lines в файл:

In [ ]:
cfg_lines = ['!',
            'service timestamps debug datetime msec localtime show-timezone year',
            'service timestamps log datetime msec localtime show-timezone year',
            'service password-encryption',
            'service sequence-numbers',
            '!',
            'no ip domain lookup',
            '!',
            'ip ssh version 2',
            '!']

cfg_lines2 = []
for line in cfg_lines:
    cfg_lines2.append(line + '\n')

f = open('r2.txt', 'w')

f.writelines(cfg_lines)
# f.writelines(cfg_lines2)
f.close()